In [2]:
import os 
import requests 
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
#Loading the openai api key
path = load_dotenv()
api_key = os.getenv("Openai_API_Key")
if api_key[:8]!="sk-proj-" : 
    print("invalide key ")
if api_key.strip()!=api_key:
    print("invalide key maybe there is a space at start and/or end")
else : 
    print("the key seem to be correct")

MODEL ="gpt-4o-mini"
openai = OpenAI()

the key seem to be correct


In [5]:
# The Website class is used for web scraping, parsing data, and extracting text and relevant links.

class website :
        """
    A class for web scraping, parsing HTML, and extracting relevant text and links from a given URL.

    Attributes:
    - url (str): The website URL.
    - title (str): The title of the webpage.
    - body (str): The raw HTML content of the page.
    - links (list[str]): A list of extracted hyperlinks from the page.
    - text (str): The cleaned text content of the webpage, excluding scripts, styles, images, and inputs.

    Methods:
    - get_contents(): Returns a formatted string containing the webpage title and text content.
    """
    url : str
    title : str
    body : str
    links : list[str]
    text : str
    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body,'html.parser')
        self.title = soup.title.text if soup.title else "there is no title"
        if soup.body:
            for irelevant in soup.body(["script","style","img","input"]):
                irelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else : 
            self.text = ""
        links = [link.get("href") for link in soup.find_all("a")]
        self.links = [link for link in links if link]
    def get_contents(self):
        return f"Webpage Title :\n{self.title}\nWebpage Contents :\n{self.text}\n\n"
        

#Use a call to gpt-4o-mini to Figure out which links are relevant

In [14]:
#Generate a system prompt
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company pagen or Careers/jobs pages.\n"
link_system_prompt +="You should respond in JSON as in this example:"
link_system_prompt += """
"links" : [
            {"type":"about page","url":"https://full.url/goes/here/about"},
            {"type":"careers page","url":"https://another.full.url/goes/here/about"}
            ]

"""

In [9]:
#Generate a user prompt 

def get_links_user_prompt(website):
        """
    Generates a user prompt for the AI model to filter relevant web links.

    Parameters:
    website (object): An object containing website details, including `url` and `links`.

    Returns:
    str: A formatted prompt listing the website links and asking the AI to select relevant ones in JSON format.
    """
    user_prompt = f"Here is the list of links on the website of {website.url}-"
    user_prompt += f"please deicde which of these are relevant web links for a brochure about company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt +="Links (some might be relative links):\n"
    user_prompt +="\n".join(website.links)
    return user_prompt

In [12]:
def get_links(url):
    """Extracts relevant links from the given website URL and formats them into a structured JSON format using an AI model.

    Parameters:
    url (str): The website URL from which links need to be extracted.

    Returns:
    dict: A JSON object containing the selected relevant links."""
    ed = website(url)
    completion = openai.chat.completions.create(
        model=MODEL,
        messages=[{"role":"system", "content":link_system_prompt},
                {"role":"user", "content":get_links_user_prompt(ed)}
                 ],
        response_format={"type":"json_object"}
    )
    result = completion.choices[0].message.content
    return json.loads(result)
    

In [16]:
get_links('https://anthropic.com')

{'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'team page', 'url': 'https://www.anthropic.com/team'},
  {'type': 'research page', 'url': 'https://www.anthropic.com/research'},
  {'type': 'news page', 'url': 'https://www.anthropic.com/news'}]}

In [36]:
def get_all_details(url): 
    result = "Landing page:\n"
    result += website(url).get_contents()
    links = get_links(url)
    for link in links["links"] : 
        result += f"\n\n{link['type']}\n"
        result += website(link['url']).get_contents()
    return result   

In [51]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [52]:
def get_brochure_user_prompt(company_name, url):
        """
    Retrieves and compiles details from a website's landing page and its relevant links.

    Parameters:
    url (str): The main website URL.

    Returns:
    str: A formatted string containing the content of the landing page followed by the extracted content from relevant links.
    """
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [134]:
def create_brochure(company_name, url):
        """
    Generates a brochure for a company using AI by processing its website content.

    Parameters:
    - company_name (str): The name of the company for which the brochure is being created.
    - url (str): The website URL of the company.

    Returns:
    None: This function streams the generated brochure content and displays it dynamically in a notebook environment.
    """
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream= True
    )
    response = ""
    display_handle = display(Markdown(""),display_id=True)
    for chunk in stream: 
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [135]:
create_brochure("HuggingFace", "https://huggingface.com")

# Hugging Face Company Brochure

## Welcome to Hugging Face!
**The AI community building the future.**  
Hugging Face is a collaborative platform designed for the machine learning community to create, discover, and share models, datasets, and applications. Our mission is to accelerate the development of artificial intelligence and empower creators and innovators in the field.

---

## Our Offerings

### **Models, Datasets, and Spaces**
- **1M+ Models:** Explore a vast repository of pre-trained machine learning models for various tasks.
- **250k+ Datasets:** A rich collection of datasets catering to different AI needs, enhancing research and development.
- **Applications:** Over 400k applications are available, empowering developers to innovate with ease.

### **Enterprise Solutions**
Hugging Face offers enterprise-grade solutions tailored for organizations, including:
- **Comprehensive Compute Options:** Deploy optimized inference endpoints; start at just $0.60/hour for GPU.
- **Dedicated Support:** Enterprise packages provide priority support, single sign-on, and secure access controls, starting at $20/user/month.

### **Open Source Community**
Our commitment to open source is evident through:
- **Transformers Library:** 139,097 implementations for Pytorch, TensorFlow, and JAX.
- **Diffusers and Tokenizers:** Powerful tools for image/audio generation and data processing, serving thousands of users.

---

## Company Culture 

At Hugging Face, we foster a *collaborative and supportive environment* where everyone is encouraged to innovate and share ideas. We believe in the power of community and strive to create a culture where **diversity** and **inclusion** are celebrated. Every team member is empowered to contribute to the collective knowledge and growth of the AI community.

---

## Customers and Collaborations

With over **50,000 organizations** utilizing our platform, Hugging Face serves a wide range of customers from startups to Fortune 500 companies including:
- **Amazon Web Services**
- **Microsoft**
- **Google**
- **Intel**

We aim to provide tools and solutions that scale across industries, enhancing productivity and advancing AI research.

---

## Careers at Hugging Face 

Join us in shaping the future of AI! At Hugging Face, we value creativity, passion, and the desire to make a difference. Our team is diverse and dedicated to pushing boundaries in machine learning. We offer various roles across different functions, right from engineering to community engagement.

### Current Openings:
Visit our careers page for available positions to become a part of our vibrant community!

---

## Connect with Us
Follow our journey and stay updated:
- [GitHub](https://github.com/huggingface)
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://www.linkedin.com/company/huggingface/)
- [Discord](https://discord.gg/huggingface)

**Hugging Face** – Together, we build the future of AI. ✨ 

--- 

For more information, visit our website: [www.huggingface.co](https://huggingface.co)

In [136]:
2+2

4